# Prof. Pedram Jahangiry

You need to make a copy to your own Google drive if you want to edit the original notebook! Start by opening this notebook on Colab 👇

<a href="https://colab.research.google.com/github/PJalgotrader/Deep_Learning-USU/blob/main/Lectures%20and%20codes/DL%20Spring%202023/Module%206-%20Deep%20Sequence%20Modeling/python/Module%206-%20NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 



![logo](https://upload.wikimedia.org/wikipedia/commons/4/44/Huntsman-Wordmark-with-USU-Blue.gif#center) 


## 🔗 Links

[![linkedin](https://img.shields.io/badge/LinkedIn-0A66C2?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pedram-jahangiry-cfa-5778015a)

[![Youtube](https://img.shields.io/badge/youtube_channel-1DA1F2?style=for-the-badge&logo=youtube&logoColor=white&color=FF0000)](https://www.youtube.com/channel/UCNDElcuuyX-2pSatVBDpJJQ)

[![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/PedramJahangiry.svg?style=social&label=Follow%20%40PedramJahangiry)](https://twitter.com/PedramJahangiry)


---


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In this part of the code, we  define a custom layer for the transformer model called **MultiHeadSelfAttention**. This layer is responsible for implementing the multi-head self-attention mechanism, a key component in the transformer architecture.

In [ ]:
# defining the transformer layer
class MultiHeadSelfAttention(layers.Layer): # Define a new class MultiHeadSelfAttention that inherits from the layers.Layer base class provided by Keras
    def __init__(self, embed_dim, num_heads=8): # The constructor for the class takes two parameters, embed_dim and num_heads. 
        # embed_dim is the dimension of the input embeddings, and num_heads is the number of attention heads in the multi-head self-attention mechanism.
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
            
        
        self.projection_dim = embed_dim // num_heads
        
        # The followings are dense (fully connected) layers responsible for computing the query, key, and value matrices, respectively, from the input embeddings.
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        # Another dense layer that combines the outputs from all attention heads.
        self.combine_heads = layers.Dense(embed_dim)

    # This method calculates the attention scores, scales them, applies the softmax function to obtain the attention weights, 
    # and then computes the output by multiplying the attention weights with the value matrix.
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32) # tf.cast is a TensorFlow function used to change the data type of a tensor. 
        # By using [-1], you are selecting the last element of the shape tensor. In this case, it corresponds to the dimension of the key vectors.
        
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output


In [ ]:
# Define the Transformer block: # https://keras.io/examples/nlp/text_classification_with_transformer/ 
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
# Positional embedding
class TokenAndPositionEmbedding(layers.Layer): # https://keras.io/api/keras_nlp/modeling_layers/token_and_position_embedding/
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

        

In [ ]:
# defining the model
def create_transformer_model(maxlen, vocab_size, embed_dim, num_heads, ff_dim, num_blocks, num_classes, dropout_rate=0.1):
    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    for _ in range(num_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(30, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
# download and prep the data
vocab_size = 20000  # Only consider the top 20k words
maxlen = 600  # Only consider the first 600 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences


In [ ]:
x_train.shape

(25000, 600)

In [19]:
# train the model
embed_dim = 32
num_heads = 2
ff_dim = 32
num_blocks = 2
num_classes = 2
dropout_rate = 0.1

model = create_transformer_model(maxlen, vocab_size, embed_dim, num_heads, ff_dim, num_blocks, num_classes, dropout_rate)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))


Epoch 1/10
782/782 [==============================] - 132s 158ms/step - loss: 0.3884 - accuracy: 0.8148 - val_loss: 0.2956 - val_accuracy: 0.8740
Epoch 2/10
782/782 [==============================] - 72s 92ms/step - loss: 0.1787 - accuracy: 0.9330 - val_loss: 0.3039 - val_accuracy: 0.8752
Epoch 3/10
782/782 [==============================] - 64s 82ms/step - loss: 0.0818 - accuracy: 0.9719 - val_loss: 0.4965 - val_accuracy: 0.8583
Epoch 4/10
782/782 [==============================] - 49s 62ms/step - loss: 0.0441 - accuracy: 0.9851 - val_loss: 0.5265 - val_accuracy: 0.8606
Epoch 5/10
782/782 [==============================] - 45s 58ms/step - loss: 0.0262 - accuracy: 0.9912 - val_loss: 0.5986 - val_accuracy: 0.8601
Epoch 6/10
782/782 [==============================] - 53s 67ms/step - loss: 0.0230 - accuracy: 0.9920 - val_loss: 0.6379 - val_accuracy: 0.8499
Epoch 7/10
782/782 [==============================] - 53s 68ms/step - loss: 0.0190 - accuracy: 0.9936 - val_loss: 0.7063 - val_accurac

# Links:
* Text classification with Transformers: https://keras.io/examples/nlp/text_classification_with_transformer/